In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import GPy
from dipy.core.gradients import gradient_table
from diGP.preprocessing_pipelines import get_SPARC_train_and_test
from diGP.dataManipulations import (DataHandler, log_q_squared)
from diGP.model import Model, get_default_kernel
from diGP.evaluation import get_SPARC_metrics

%matplotlib inline
sns.set_style('dark')

In [ ]:
with open('../config.json', 'r') as json_file:
    conf = json.load(json_file)
data_paths = conf['SPARC']['data_paths']
q_test_path = conf['SPARC']['q_test_path']

Load data to use for prediction.

In [ ]:
source = 'gradient_20'
gtab, data, voxelSize = get_SPARC_train_and_test(data_paths[source], data_paths['goldstandard'], q_test_path)

In [ ]:
lmbda=None
qMagnitudeTransform=None
handler = DataHandler(gtab['train'], data['train'], qMagnitudeTransform=qMagnitudeTransform,
                      voxelSize=voxelSize[0:2], box_cox_lambda=lmbda)
handlerPred = DataHandler(gtab['test'], data=None, spatial_shape=data['test'].shape[0:2],
                          qMagnitudeTransform=qMagnitudeTransform, voxelSize=voxelSize[0:2], box_cox_lambda=lmbda)


In [ ]:
kernel = get_default_kernel(handler)

grid_dims = [[0], [1], [2, 3, 4, 5]]

model = Model(handler, kernel, data_handler_pred=handlerPred, grid_dims=grid_dims, verbose=False)

In [ ]:
np.random.seed(0)
model.train(restarts=False)

print(model.GP_model)
print("\nLegendre coefficients: \n{}".format(model.GP_model.mul.LegendrePolynomial.coefficients))

In [ ]:
mu = model.predict(compute_var=False)

pred = {'GP': mu}

## Reference prediction with dipy

In [ ]:
import dipy.reconst.dti as dti

tenmodel = dti.TensorModel(gtab['train'])
tenfit = tenmodel.fit(data['train'])

pred['DTI'] = tenfit.predict(gtab['test'])

In [ ]:
import dipy.reconst.mapmri as mapmri

map_model = mapmri.MapmriModel(gtab['train'], positivity_constraint=False, laplacian_weighting='GCV',
                               radial_order=6)
mapfit = map_model.fit(data['train'])

pred['MAP'] = mapfit.predict(gtab['test'])

## Evaluation

In [ ]:
idx = 50 * np.arange(9)
f, axs = plt.subplots(len(idx), 4)
f.set_figheight(len(idx)*2)
f.set_figwidth(10)
for i, index in enumerate(idx):
    axs[i, 0].imshow(data['test'][:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 0].axis('off')
    axs[i, 0].set_title('Ground truth,\nb = {}'.format(np.round(gtab['test'].bvals[index])))
    axs[i, 1].imshow(pred['GP'][:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 1].axis('off')
    axs[i, 1].set_title('GP prediction, \nb = {}'.format(np.round(gtab['test'].bvals[index])))
    axs[i, 2].imshow(pred['DTI'][:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 2].axis('off')
    axs[i, 2].set_title('DTI prediction, \nb = {}'.format(np.round(gtab['test'].bvals[index])))
    axs[i, 3].imshow(pred['MAP'][:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 3].axis('off')
    axs[i, 3].set_title('MAP-MRI prediction, \nb = {}'.format(np.round(gtab['test'].bvals[index])))

plt.tight_layout()

In [ ]:
for key, value in pred.items():
    print("\n{} model:".format(key))
    get_SPARC_metrics(gtab['test'], data['test'], value, verbose=True)